<a href="https://colab.research.google.com/github/yyyyyokoko/590FaceMaskDetection/blob/master/Mask_Detector_CNN_Julia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# number of files (images) that are in each folder
# #train
# print("The number of images labelled 'WithMask' in the training set:",len(os.listdir('Train/WithMask')))
# print("The number of images labelled 'WithoutMask' in the training set:",len(os.listdir('Train/WithoutMask')))

# #test
# print("The number of images labelled 'WithMask' in the testing set:",len(os.listdir('Test/WithMask')))
# print("The number of images labelled 'WithoutMask' in the testing set:",len(os.listdir('Test/WithoutMask')))

# #val
# print("The number of images labelled 'WithMask' in the val set:",len(os.listdir('Validation/WithMask')))
# print("The number of images labelled 'WithoutMask' in the val set:",len(os.listdir('Validation/WithoutMask')))

The number of images labelled 'WithMask' in the training set: 5000
The number of images labelled 'WithoutMask' in the training set: 5000
The number of images labelled 'WithMask' in the testing set: 483
The number of images labelled 'WithoutMask' in the testing set: 509
The number of images labelled 'WithMask' in the val set: 400
The number of images labelled 'WithoutMask' in the val set: 400


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(100, (3,3), activation='relu', input_shape=(160, 160, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(100, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])


model.compile(optimizer="adam",loss="categorical_crossentropy",metrics ="accuracy")


In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 158, 158, 100)     2800      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 79, 79, 100)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 77, 77, 100)       90100     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 38, 38, 100)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 144400)            0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 144400)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 50)               

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# train_dir = '/content/drive/My Drive/ColabNotebooks/Face Mask Dataset/Train/'
train_dir = '/content/drive/My Drive/ColabNotebooks/Face Mask Dataset/Validation/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# train_dir = "C:/Users/zhuyi/Documents/ANLY590_DEEP/project/Train"
train_data = ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.15,
        rescale=1./255,
        horizontal_flip=True,
        validation_split=0.2)

train = train_data.flow_from_directory(directory=train_dir,target_size=(160,160),
                                          class_mode="categorical",batch_size=32,subset = "training")

valid_datagen = ImageDataGenerator(rescale=1./255)
valid = train_data.flow_from_directory(directory=train_dir,target_size=(160,160),
                                          class_mode="categorical",batch_size=32,subset="validation")

# train_generator = train_data.flow_from_directory(train_dir,batch_size=10,target_size=(160, 160))

Found 526 images belonging to 2 classes.
Found 130 images belonging to 2 classes.


In [ ]:
# val_dir = '/content/drive/My Drive/ColabNotebooks/Face Mask Dataset/Validation/'
# val_data = ImageDataGenerator(rescale=1.0/255)
# val_generator = val_data.flow_from_directory(val_dir,batch_size=10,target_size=(160, 160))

In [ ]:
# checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
checkpoint = ModelCheckpoint("moblenet_facemask.h5",monitor="val_accuracy",save_best_only=True,verbose=1)


In [ ]:
# history = model.fit_generator(train_generator,
#                               epochs=10,
#                               validation_data=val_generator,
#                               callbacks=[checkpoint])


history = model.fit_generator(generator=train, validation_data=valid,
                               callbacks=[checkpoint],
                              epochs=5)



Epoch 1/10
17/17 [==============================] - ETA: 0s - loss: 0.2161 - accuracy: 0.9183
Epoch 00001: val_accuracy improved from -inf to 0.91538, saving model to moblenet_facemask.h5
17/17 [==============================] - 61s 4s/step - loss: 0.2161 - accuracy: 0.9183 - val_loss: 0.1930 - val_accuracy: 0.9154
Epoch 2/10
17/17 [==============================] - ETA: 0s - loss: 0.1381 - accuracy: 0.9563
Epoch 00002: val_accuracy improved from 0.91538 to 0.94615, saving model to moblenet_facemask.h5
17/17 [==============================] - 60s 4s/step - loss: 0.1381 - accuracy: 0.9563 - val_loss: 0.1869 - val_accuracy: 0.9462
Epoch 3/10
17/17 [==============================] - ETA: 0s - loss: 0.1133 - accuracy: 0.9677
Epoch 00003: val_accuracy did not improve from 0.94615
17/17 [==============================] - 60s 4s/step - loss: 0.1133 - accuracy: 0.9677 - val_loss: 0.2272 - val_accuracy: 0.9308
Epoch 4/10
17/17 [==============================] - ETA: 0s - loss: 0.1138 - accuracy

In [ ]:
model.evaluate_generator(valid)


Instructions for updating:
Please use Model.evaluate, which supports generators.


[0.16063416004180908, 0.9538461565971375]

In [ ]:
pred = model.predict_classes(valid)
pred[:15]

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0])

In [ ]:
# model.save(os.path.join("C:/Users/zhuyi/Documents/ANLY590_DEEP/project",'model.h5'))
model.save("/content/drive/My Drive/ColabNotebooks/face_mask_julia_cnn_new.h5")


In [ ]:
import cv2

In [ ]:
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
labels_dict={0:'without_mask',1:'with_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
webcam = cv2.VideoCapture(0) #Use camera 0

# We load the xml file
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror

    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        #print(result)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Show the image
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-6sxsq0tp\opencv\modules\objdetect\src\cascadedetect.cpp:1689: error: (-215:Assertion failed) !empty() in function 'cv::CascadeClassifier::detectMultiScale'


In [ ]:
webcam.release()

In [ ]:
cv2.destroyAllWindows()